In [3]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [4]:
log_name = 'test'
with open('../transformed_event_logs/BPIC_2018_start_end_test.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['00037f','0087cf','019209','023bb9','03b214','059abc','07361d','08e484','0;n/a','0d884f','0fe39b','13c937','155add','167893','1bd3b5','1c1894','2044b6','21612d','237892','27cc37','28b6bf','2ac4ac','2b8616','2baab0','2bf205','2c546f','2ca0ae','2dc625','313338','346f05','352a49','354865','36c75c','39be3e','3d952e','40178c','425ee3','4298e3','439089','44798d','465290','46e3af','478c4f','483029','4af6fb','4b9a7f','4e9bc2','4fac6d','51d239','520882','556da9','5a21a5','5bf77a','5c1b3c','5d8173','5d9e09','5dd4ec','5e018b','60b0b8','65bf34','6ad4a6','6c720c','6d6ae5','6f6a4e','6fde6b','7078c7','71ffe8','727350','75992a','76f30e','7850e8','7886b1','79367e','796771','7b9b55','7d03a0','7d12ba','7d4a25','7fb8a5','815d19','822fb7','82518f','83c7b7','8a1fba','8aab8f','8beb64','8c9a01','8d8538','91bca0','9765da','97d224','99e048','9aaf68','9e088c','9e337f','DP-R','DP-Z','Document processing automaton','Inspection automaton','Inspection service','Notification automaton','Parcel automaton','Processing automaton','Reference alignment processor','Remote inspection export','Remote inspection import','a21e1b','a5ae3f','a5c3dd','abe845','ad25fc','aea7c8','af1e1e','b2b786','b33ed5','b4755f','b90293','ba978b','bbd307','bc063d','bfa0ec','c0aa4a','c39cb2','c4f075','c522f8','c70c37','c70d79','c7c478','c966f4','c98a4a','d0f451','d114c8','d4758f','d4d37d','d85681','d8639c','dd04e2','dd12cf','dde669','dee71b','dfd1d4','e25f20','e99397','ea52aa','eb38eb','eb7015','ee262f','ee28de','ef80e3','f01c35','f3e81e','f48280','f6846a','f6f7f1','f7f7b6','f8da29','f9fe07','fa6437','fa68d2','fb5fa8','fc6177','fcb55b','ffb56b','parcel correction automaton','scheduler']
known_activities = ['abort external','abort payment','approve','begin admissibility check','begin editing','begin editing from refused','begin payment','begin preparations','calculate','calculate protocol','cancel offline','change department','check','check admissibility','clear','correction GFM17','create','decide','discard','finish editing','finish payment','finish pre-check','finish preparations','initialize','insert document','mail income','mail valid','performed','performed offline','plan','prepare external','prepare offline','refuse','remove document','restart editing','revoke approval','revoke decision','revoke withdrawal','save','take original document','withdraw']

/tmp/ipykernel_797022/4066369457.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [5]:
test_event_log

,success_start,org:resource_start,docid_uuid_start,doctype_start,subprocess_start,docid_start,activity_start,note_start,eventid_start,identity:id_start,concept:name_start,lifecycle:transition_start,time:timestamp_start,case:young farmer_start,case:selected_random_start,case:penalty_AJLP_start,case:application_start,case:penalty_amount0_start,case:program-id_start,case:penalty_BGKV_start,case:penalty_AUVP_start,case:applicant_start,case:risk_factor_start,case:small farmer_start,case:penalty_BGP_start,case:department_start,case:penalty_C16_start,case:penalty_BGK_start,case:penalty_AVUVP_start,case:penalty_CC_start,case:penalty_AVJLP_start,case:penalty_C9_start,case:cross_compliance_start,case:rejected_start,case:greening_start,case:penalty_C4_start,case:penalty_AVGP_start,case:penalty_ABP_start,case:penalty_B6_start,case:penalty_B4_start,case:penalty_B5_start,case:penalty_AVBP_start,case:penalty_B2_start,case:selected_risk_start,case:penalty_B3_start,case:area_start,case:selected_manually_start,case:penalty_AGP_start,case:penalty_B16_start,case:penalty_GP1_start,case:basic payment_start,case:penalty_B5F_start,case:penalty_V5_start,case:payment_actual0_start,case:identity:id_start,case:amount_applied0_start,case:redistribution_start,case:penalty_JLP6_start,case:penalty_JLP7_start,case:year_start,case:penalty_JLP5_start,case:penalty_JLP2_start,case:penalty_JLP3_start,case:number_parcels_start,case:penalty_JLP1_start,case:concept:name,case:penalty_amount1_start,case:payment_actual1_start,case:amount_applied1_start,case:penalty_amount2_start,case:payment_actual2_start,case:amount_applied2_start,case:penalty_amount3_start,case:payment_actual3_start,case:amount_applied3_start,success_complete,org:resource_complete,docid_uuid_complete,doctype_complete,subprocess_complete,docid_complete,activity_complete,note_complete,eventid_complete,identity:id_complete,concept:name_complete,lifecycle:transition_complete,time:timestamp_complete,case:young farmer_complete,case:selected_random_complete,case:penalty_AJLP_complete,case:application_complete,case:penalty_amount0_complete,case:program-id_complete,case:penalty_BGKV_complete,case:penalty_AUVP_complete,case:applicant_complete,case:risk_factor_complete,case:small farmer_complete,case:penalty_BGP_complete,case:department_complete,case:penalty_C16_complete,case:penalty_BGK_complete,case:penalty_AVUVP_complete,case:penalty_CC_complete,case:penalty_AVJLP_complete,case:penalty_C9_complete,case:cross_compliance_complete,case:rejected_complete,case:greening_complete,case:penalty_C4_complete,case:penalty_AVGP_complete,case:penalty_ABP_complete,case:penalty_B6_complete,case:penalty_B4_complete,case:penalty_B5_complete,case:penalty_AVBP_complete,case:penalty_B2_complete,case:selected_risk_complete,case:penalty_B3_complete,case:area_complete,case:selected_manually_complete,case:penalty_AGP_complete,case:penalty_B16_complete,case:penalty_GP1_complete,case:basic payment_complete,case:penalty_B5F_complete,case:penalty_V5_complete,case:payment_actual0_complete,case:identity:id_complete,case:amount_applied0_complete,case:redistribution_complete,case:penalty_JLP6_complete,case:penalty_JLP7_complete,case:year_complete,case:penalty_JLP5_complete,case:penalty_JLP2_complete,case:penalty_JLP3_complete,case:number_parcels_complete,case:penalty_JLP1_complete,case:penalty_amount1_complete,case:payment_actual1_complete,case:amount_applied1_complete,case:penalty_amount2_complete,case:payment_actual2_complete,case:amount_applied2_complete,case:penalty_amount3_complete,case:payment_actual3_complete,case:amount_applied3_complete,duration,duration_seconds,duration_ms,duration_hours,seconds_in_day,day_of_week,abort external,abort payment,approve,begin admissibility check,begin editing,begin editing from refused,begin payment,begin preparations,calculate,calculate protocol,cancel offline,change department,check,check admissibility,clear,correction GFM17,create,decide,discard,finish editing,finish payment,finish pre-check,finish

In [6]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [7]:
drbart_model_A = DRBART(parser_dir = '../../../models/bpic_2018/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A = evaluator_A.sample_cases(False, False)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2018/concept-name/ucuts.json'

  0%|▏                                                                    | 2/892 [00:12<1:19:57,  5.39s/it]

  0%|▎                                                                      | 4/892 [00:13<34:36,  2.34s/it]

  1%|▋                                                                      | 8/892 [00:13<13:31,  1.09it/s]

  1%|▊                                                                     | 10/892 [00:16<15:48,  1.08s/it]

...

 99%|████████████████████████████████████████████████████████████████████ | 880/892 [06:21<00:01,  6.22it/s]

 99%|████████████████████████████████████████████████████████████████████▎| 883/892 [06:21<00:01,  6.76it/s]

 99%|████████████████████████████████████████████████████████████████████▌| 886/892 [06:22<00:01,  4.80it/s]

100%|████████████████████████████████████████████████████████████████████▊| 890/892 [06:23<00:00,  5.16it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [06:23<00:00,  2.33it/s]

In [7]:
sum([v.ln() for v in likelihoods_A[0].values()])

Decimal('-12394.39609967806777429331935')

In [8]:
np.mean(get_pscores(likelihoods_A))

np.float64(10125.258481097446)

In [9]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/bpic_2018/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R = evaluator_A_R.sample_cases(False, False)

ValueError: invalid literal for int() with base 10: ''

  1%|▉                                                                     | 12/892 [00:11<09:19,  1.57it/s]

  2%|█▎                                                                    | 17/892 [00:13<07:51,  1.85it/s]

  2%|█▌                                                                    | 20/892 [00:14<06:34,  2.21it/s]

  3%|██                                                                    | 27/892 [00:16<05:10,  2.79it/s]

...

 99%|████████████████████████████████████████████████████████████████████▏| 881/892 [05:07<00:01,  6.15it/s]

 99%|████████████████████████████████████████████████████████████████████▍| 884/892 [05:08<00:01,  4.52it/s]

 99%|████████████████████████████████████████████████████████████████████▌| 887/892 [05:09<00:00,  5.43it/s]

100%|████████████████████████████████████████████████████████████████████▉| 891/892 [05:09<00:00,  5.86it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [05:09<00:00,  2.88it/s]

In [10]:
sum([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('-10605.75156558240907385539443')

In [11]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(9884.028197417258)

In [15]:
drbart_model_R = DRBART(parser_dir = '../../../models/bpic_2018/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name_start',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                                               | 0/892 [00:00<?, ?it/s]

  0%|                                                                     | 1/892 [00:15<3:52:18, 15.64s/it]

  0%|▏                                                                    | 2/892 [00:18<2:03:39,  8.34s/it]

  0%|▎                                                                      | 4/892 [00:21<56:24,  3.81s/it]

  1%|▋                                                                      | 9/892 [00:22<19:57,  1.36s/it]

...

 95%|█████████████████████████████████████████████████████████████████▎   | 844/892 [08:01<00:07,  6.23it/s]

 97%|██████████████████████████████████████████████████████████████████▊  | 864/892 [08:02<00:02, 11.12it/s]

 98%|███████████████████████████████████████████████████████████████████▋ | 875/892 [08:03<00:01, 10.06it/s]

 99%|████████████████████████████████████████████████████████████████████▌| 887/892 [08:03<00:00, 13.41it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [08:03<00:00,  1.84it/s]

In [16]:
sum([v.ln() for v in likelihoods_R[0].values()])

Decimal('-9979.051025463528732293692787')

In [17]:
np.mean(get_pscores(likelihoods_R))

np.float64(10347.619780187357)

In [10]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/bpic_2018/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name_start',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

ValueError: invalid literal for int() with base 10: ''

  0%|▎                                                                      | 4/892 [00:10<31:37,  2.14s/it]

  1%|▊                                                                     | 10/892 [00:11<11:02,  1.33it/s]

  1%|█                                                                     | 13/892 [00:11<07:50,  1.87it/s]

  2%|█▎                                                                    | 16/892 [00:11<05:41,  2.56it/s]

...

 99%|████████████████████████████████████████████████████████████████████▍| 885/892 [04:50<00:01,  5.34it/s]

 99%|████████████████████████████████████████████████████████████████████▌| 887/892 [04:50<00:00,  5.26it/s]

100%|████████████████████████████████████████████████████████████████████▋| 888/892 [04:50<00:00,  4.87it/s]

100%|████████████████████████████████████████████████████████████████████▉| 891/892 [04:50<00:00,  6.63it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [04:50<00:00,  3.07it/s]

In [19]:
sum([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('-10131.81903188291264094819622')

In [20]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(9682.308662541092)

In [11]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/bpic_2018/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name_start',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

ValueError: invalid literal for int() with base 10: ''

  0%|▎                                                                      | 4/892 [00:06<19:26,  1.31s/it]

  1%|▍                                                                      | 5/892 [00:06<14:56,  1.01s/it]

  1%|▋                                                                      | 9/892 [00:07<06:14,  2.36it/s]

  1%|█                                                                     | 13/892 [00:07<04:28,  3.27it/s]

...

 99%|████████████████████████████████████████████████████████████████████▏| 881/892 [04:07<00:01,  5.51it/s]

 99%|████████████████████████████████████████████████████████████████████▍| 885/892 [04:08<00:01,  5.66it/s]

100%|████████████████████████████████████████████████████████████████████▊| 889/892 [04:08<00:00,  8.00it/s]

100%|████████████████████████████████████████████████████████████████████▉| 891/892 [04:09<00:00,  6.11it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [04:09<00:00,  3.58it/s]

In [22]:
sum([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('-10337.28306961683828713270933')

In [23]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(9643.317638246413)

In [24]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|                                                                               | 0/892 [00:00<?, ?it/s]

  0%|                                                                     | 1/892 [00:06<1:30:16,  6.08s/it]

  0%|▏                                                                      | 2/892 [00:06<41:22,  2.79s/it]

  0%|▎                                                                      | 4/892 [00:07<19:27,  1.31s/it]

  1%|▋                                                                      | 8/892 [00:08<08:38,  1.70it/s]

...

 99%|████████████████████████████████████████████████████████████████████ | 880/892 [03:34<00:01,  8.15it/s]

 99%|████████████████████████████████████████████████████████████████████▎| 883/892 [03:34<00:00,  9.91it/s]

 99%|████████████████████████████████████████████████████████████████████▌| 887/892 [03:35<00:00,  9.56it/s]

100%|████████████████████████████████████████████████████████████████████▊| 889/892 [03:35<00:00,  6.91it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [03:35<00:00,  4.13it/s]

In [25]:
sum([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('-10001.49968523186357705596833')

In [26]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(8284.12879800822)

In [27]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/artificial/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|                                                                               | 0/892 [00:00<?, ?it/s]

  0%|                                                                     | 1/892 [00:05<1:16:05,  5.12s/it]

  0%|▏                                                                      | 3/892 [00:05<22:05,  1.49s/it]

  1%|▍                                                                      | 6/892 [00:05<09:13,  1.60it/s]

  1%|▋                                                                      | 8/892 [00:06<06:27,  2.28it/s]

...

 99%|████████████████████████████████████████████████████████████████████▏| 882/892 [03:42<00:01,  5.83it/s]

 99%|████████████████████████████████████████████████████████████████████▎| 883/892 [03:42<00:01,  6.09it/s]

100%|████████████████████████████████████████████████████████████████████▋| 888/892 [03:42<00:00,  7.53it/s]

100%|████████████████████████████████████████████████████████████████████▉| 891/892 [03:43<00:00,  6.43it/s]

100%|█████████████████████████████████████████████████████████████████████| 892/892 [03:43<00:00,  3.99it/s]

In [28]:
sum([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('-9827.326635781784837518068833')

In [29]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(8126.961361848348)

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./artificial_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)